# Word Vector Representation
## Word Meaning
Meaning is defined in terms of vectors. We will build a dense vector for each word type, chosen so that it is good at predicting other words appearing in its context.
```python
linguistics = [0.286, 0.792, -0.177, -0.107, 0.109, -0.542]
```

We define a model that aims to predict between a center word $w_{t}$ and context words in terms of word vectors.
$$
P(\text{Context}\mid w_{t}) = ... 
$$
which has a loss function, e.g.
$$
J = 1 - P(w_{-t}\mid w_{t})
$$

We look at many positions `t` in a big language corpus. We keep adjusting the vector representation of words to minimize this loss.

### word2vec
For each word `t = 1 ... T`, predict surrounding words in a window of radius `m` of every word. The objective is to maximize the probability of any context word given the current center word.
$$
J^{\prime}(\theta) = \prod_{t=1}^{T} \prod_{-m \leq j \lt m} P(w_{t+j} \mid w_{t}; \theta)
$$

It's easier to work with summation so we turn the probabilities into log probabilities. People tend to prefer minimization over maximization, so we put a negative sign in front.
$$
J(\theta) = \frac{-1}{T} \sum_{t=1}^{T} \sum_{-m \leq j \lt m} log \; P(w_{t+j} \mid w_{t}; \theta)
$$